In [ ]:
import numpy as np
import pandas as pd

train = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv", index_col="essay_id")
test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv", index_col="essay_id")

In [ ]:
# get essay length (except spaces and e)
# and bin into 50s
train["length"] = train.full_text.str.replace("[Ee\s]", "", regex=True).str.len() // 50
test["length"] = test.full_text.str.replace("[Ee\s]", "", regex=True).str.len() // 50

# Get modal score per length bin
# assume monotone-increasing
modes = (train
    .groupby("length")
    .score
    .agg(lambda s: s.value_counts().keys()[0])
    .sort_index()
    .reindex(range(0, 200))
    .fillna(0)
    .clip(lower=1)
    .cummax()
    .astype(int))

modes.plot.line(title="Modal score by length bin\n(monotonicity assumed)")
print("MAE:", (train.length.map(modes) - train.score).abs().mean())

In [ ]:
submission = test.length.map(modes).rename("score")
submission.to_csv("submission.csv")
submission